In [1]:
import dotenv

dotenv.load_dotenv()

True

## Assemble RAG Pipeline

In [2]:
from rag import weaviate_utils
from llama_index import VectorStoreIndex

# assumes that the database has already been populated
weaviate_client = weaviate_utils.get_weaviate_client()
vector_store = weaviate_utils.as_vector_store(weaviate_client, class_name="DocsChunk")
index = VectorStoreIndex.from_vector_store(vector_store)

2024-02-12 11:16:00.736 
  command:

    streamlit run /home/kangonaut/data/aegis-athena/venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
/home/kangonaut/data/aegis-athena/venv/lib/python3.11/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(
/home/kangonaut/data/aegis-athena/venv/lib/python3.11/site-packages/weaviate/__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


In [14]:
query_engine = index.as_query_engine()

In [15]:
response = query_engine.query("What does EPS stand for?")
print(str(response))

EPS stands for Electrical Power System.


## Evaluation

In [16]:
from trulens_eval import Tru
tru = Tru()

In [17]:
from trulens_eval.feedback.provider.openai import OpenAI
openai = OpenAI()

In [18]:
from trulens_eval.app import App

# select context to be used in feedback
context = App.select_context(query_engine)

In [19]:
from trulens_eval.feedback import Groundedness, Feedback

grounded = Groundedness(groundedness_provider=OpenAI())
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [9]:
# response relevance
f_qa_relevance = Feedback(openai.relevance, name="Response Relevance").on_input_output()

✅ In Response Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Response Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [20]:
import numpy as np

# context relevance
f_qs_relevance = (
    Feedback(openai.qs_relevance, name="Context Relevance")
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [21]:
from trulens_eval import TruLlama

tru_query_engine_recorder = TruLlama(
    app=query_engine,
    app_id="basic-1",
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance]
)

In [22]:
with tru_query_engine_recorder as recording:
    query_engine.query("How can I turn off the ECS controller?")

In [24]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.178.44:8501


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [25]:
tru.stop_dashboard()

In [26]:
tru.reset_database()